<h4><b>Обучение модели с помощью логистической регрессии и случайного леса:</b></h4>

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder \
    .appName("data analitic") \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv("DataFrame.csv")
df.show()


+---+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+
|_c0|income_type|children|     days_employed|dob_years|education_id|family_status_id|gender|debt|total_income|             purpose|total_income_category|    purpose_category|
+---+-----------+--------+------------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+
|  0|  компаньон|       0| 191.1467457844417|       64|           0|               1|     F|   0|      186408|             свадьба|                    C|  проведение свадьбы|
|  1|  компаньон|       0| 563.9370284307207|       33|           1|               0|     M|   0|      161976|на покупку своего...|                    C|операции с автомо...|
|  2|  компаньон|       2|1653.2725398991834|       25|           0|               0|     F|   0|      102955|          автом

<h4><b>Предворительная обработка данных:</b></h4>

Преобразовывание данных ДатаФрейма в целочисленный тип:

In [14]:
df = df.withColumn("children", df["children"].cast(IntegerType()))
df = df.withColumn("dob_years", df["dob_years"].cast(IntegerType()))
df = df.withColumn("education_id", df["education_id"].cast(IntegerType()))
df = df.withColumn("family_status_id", df["family_status_id"].cast(IntegerType()))
df = df.withColumn("debt", df["debt"].cast(IntegerType()))
df = df.withColumn("total_income", df["total_income"].cast(IntegerType()))
df = df.withColumn("days_employed", df["days_employed"].cast(IntegerType()))
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- income_type: string (nullable = true)
 |-- children: integer (nullable = true)
 |-- days_employed: integer (nullable = true)
 |-- dob_years: integer (nullable = true)
 |-- education_id: integer (nullable = true)
 |-- family_status_id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- debt: integer (nullable = true)
 |-- total_income: integer (nullable = true)
 |-- purpose: string (nullable = true)
 |-- total_income_category: string (nullable = true)
 |-- purpose_category: string (nullable = true)



Преобразование категориальных столбцов в числовые с помощью `StringIndexer` и Создание объекта `VectorAssembler` для объединения признаков в один столбец 

In [30]:
# Преобразовываем категориальные столбцы в числовые с помощью StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in ['income_type', 'gender', 'purpose_category']]

# Создаем объект VectorAssembler для объединения признаков в один столбец 'features'
assembler = VectorAssembler(
    inputCols=['income_type_index', 'children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender_index', 'debt', 'total_income', 'purpose_category_index'],
    outputCol="features")

# Применяем StringIndexer и VectorAssembler к датафрейму
indexed_df = df
for i in indexers:
    indexed_df = i.transform(indexed_df)
indexed_df = assembler.transform(indexed_df)

indexed_df.show()

+---+-----------+--------+-------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+-----------------+------------+----------------------+--------------------+
|_c0|income_type|children|days_employed|dob_years|education_id|family_status_id|gender|debt|total_income|             purpose|total_income_category|    purpose_category|income_type_index|gender_index|purpose_category_index|            features|
+---+-----------+--------+-------------+---------+------------+----------------+------+----+------------+--------------------+---------------------+--------------------+-----------------+------------+----------------------+--------------------+
|  0|  компаньон|       0|          191|       64|           0|               1|     F|   0|      186408|             свадьба|                    C|  проведение свадьбы|              1.0|         0.0|                   3.0|[1.0,0.0,191.0,64...|
|  1|  компаньон|   

Разделение на обучающий и тестовый наборы в пропорции 70% на 60% соответсвенно

In [ ]:
(training_data, test_data) = indexed_df.randomSplit([0.7, 0.3], seed=12345)

<h4><b>Обучение модели с помощью логистической регрессии</b></h4>

In [28]:
from pyspark.ml.classification import LogisticRegression

# Создаем модель логистической регрессии
lr = LogisticRegression(labelCol="debt", featuresCol="features", maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Обучаем модель на обучающем наборе
lr_model = lr.fit(training_data)

# Применяем модель к тестовому набору
predictions = lr_model.transform(test_data)

# Оцениваем качество модели
evaluator = MulticlassClassificationEvaluator(labelCol="debt", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)



Accuracy: 0.9201964395334561


<h4><b>Обучение модели с помощью случайного леса</b></h4>

In [29]:
from pyspark.ml.classification import RandomForestClassifier

# Создаем модель случайного леса
rf = RandomForestClassifier(labelCol="debt", featuresCol="features", numTrees=100)

# Обучаем модель на обучающем наборе
rf_model = rf.fit(training_data)

# Применяем модель к тестовому набору
predictions = rf_model.transform(test_data)

# Оцениваем качество модели
evaluator = MulticlassClassificationEvaluator(labelCol="debt", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 1.0
